# Quelle des Codes
> https://machinelearningmastery.com/simulated-annealing-from-scratch-in-python/

In [7]:
from numpy import asarray
from numpy import exp
from numpy.random import randn
from numpy.random import rand
from numpy.random import seed

# objective function
def objective(x):
	return x[0]**2.0

# simulated annealing algorithm
def simulated_annealing(objective, bounds, n_iterations, step_size, temp):
	# generate an initial point
	best = bounds[:, 0] + rand(len(bounds)) * (bounds[:, 1] - bounds[:, 0])
	# evaluate the initial point
	best_eval = objective(best)
	# current working solution
	curr, curr_eval = best, best_eval
	# run the algorithm
	for i in range(n_iterations):
		# take a step
		candidate = curr + randn(len(bounds)) * step_size
		# evaluate candidate point
		candidate_eval = objective(candidate)
		# check for new best solution
		if candidate_eval < best_eval:
			# store new best point
			best, best_eval = candidate, candidate_eval
			# report progress
			print('>%d f(%s) = %.5f' % (i, best, best_eval))
		# difference between candidate and current point evaluation
		diff = candidate_eval - curr_eval
		# calculate temperature for current epoch
		t = temp / float(i + 1)
		# calculate metropolis acceptance criterion
		metropolis = exp(-diff / t)
		# check if we should keep the new point
		if diff < 0 or rand() < metropolis:
			# store the new current point
			curr, curr_eval = candidate, candidate_eval
	return [best, best_eval]

# seed the pseudorandom number generator
seed(1)
# define range for input
bounds = asarray([[-5.0, 5.0]])
# define the total iterations
n_iterations = 1000
# define the maximum step size
step_size = 0.1
# initial temperature
temp = 10
# perform the simulated annealing search
best, score = simulated_annealing(objective, bounds, n_iterations, step_size, temp)
print('Done!')
print('f(%s) = %f' % (best, score))

>34 f([-0.78753544]) = 0.62021
>35 f([-0.76914239]) = 0.59158
>37 f([-0.68574854]) = 0.47025
>39 f([-0.64797564]) = 0.41987
>40 f([-0.58914623]) = 0.34709
>41 f([-0.55446029]) = 0.30743
>42 f([-0.41775702]) = 0.17452
>43 f([-0.35038542]) = 0.12277
>50 f([-0.15799045]) = 0.02496
>66 f([-0.11089772]) = 0.01230
>67 f([-0.09238208]) = 0.00853
>72 f([-0.09145261]) = 0.00836
>75 f([-0.05129162]) = 0.00263
>93 f([-0.02854417]) = 0.00081
>144 f([0.00864136]) = 0.00007
>149 f([0.00753953]) = 0.00006
>167 f([-0.00640394]) = 0.00004
>225 f([-0.00044965]) = 0.00000
>503 f([-0.00036261]) = 0.00000
>512 f([0.00013605]) = 0.00000
Done!
f([0.00013605]) = 0.000000


# Quelle
> https://pythonmana.com/2021/05/20210502184221296q.html
>
> **https://ofstack.com/python/45529/python-mathematical-modeling-learning-simulated-annealing-algorithm-example-analysis-of-integer-programming-problem.html**

In [5]:
# Simulated annealing algorithm Program ： Solving linear programming problems （ Integer programming ）

# Program: SimulatedAnnealing_v4.py

# Purpose: Simulated annealing algorithm for function optimization

# v4.0: Integer programming ： The value of decision variable is integer （ Both the initial value and the new solution are randomly generated integers ）

# Copyright 2021 YouCans, XUPT

# Crated：2021-05-01
# -*- coding: utf-8 -*-
import math # The import module
import random # The import module
import pandas as pd # The import module YouCans, XUPT
import numpy as np # The import module numpy, And abbreviated as np
import matplotlib.pyplot as plt
from datetime import datetime

# Subroutines ： Define the objective function of the optimization problem
def cal_Energy(X: list[int], nVar: int, mk: int) -> float: # m(k)： Punishment factor , With the number of iterations k Gradually increase
    p1 = (max(0, 6*X[0]+5*X[1]-60))**2
    p2 = (max(0, 10*X[0]+20*X[1]-150))**2
    fx = -(10*X[0]+9*X[1])
    return fx+mk*(p1+p2)

# Subroutines ： Parameter setting of simulated annealing algorithm
def ParameterSetting():
    cName = "funcOpt" # Define problem name YouCans, XUPT
    nVar = 2 # Given the number of arguments ,y=f(x1,..xn)
    xMin = [0, 0] # Given the lower bound of the search space ,x1_min,..xn_min
    xMax = [8, 8] # Given the upper limit of the search space ,x1_max,..xn_max
    tInitial = 100.0 # Set the initial annealing temperature (initial temperature)
    tFinal = 1 # Set the ending annealing temperature (stop temperature)
    alfa = 0.98 # Set the cooling parameters ,T(k)=alfa*T(k-1)
    meanMarkov = 100 # Markov Chain length , That is, the number of internal circulation runs
    scale = 0.5 # Define the search step size , It can be set to a fixed value or gradually reduced
    return cName, nVar, xMin, xMax, tInitial, tFinal, alfa, meanMarkov, scale

# Simulated annealing algorithm
def OptimizationSSA(nVar: int,xMin: int,xMax: int,tInitial: float,tFinal: float,alfa: float,meanMarkov: 100,scale: float):
    # ====== Initialize the random number generator ======
    randseed = random.randint(1, 100)
    random.seed(randseed) # Random number generator set seed , It can also be set to a specified integer
    # ====== The initial solution of the optimization problem is generated randomly ======
    xInitial = np.zeros((nVar)) # initialization , Create array
    for v in range(nVar):
        # xInitial[v] = random.uniform(xMin[v], xMax[v]) # produce [xMin, xMax] Random real numbers of ranges
        xInitial[v] = random.randint(xMin[v], xMax[v]) # produce [xMin, xMax] The random integer of the range
    # Call subfunction cal_Energy Calculate the objective function value of the current solution
    fxInitial = cal_Energy(xInitial, nVar, 1) # m(k)： Punishment factor , The initial value is 1
    # ====== Simulated annealing algorithm initialization ======
    xNew = np.zeros((nVar)) # initialization , Create array
    xNow = np.zeros((nVar)) # initialization , Create array
    xBest = np.zeros((nVar)) # initialization , Create array
    xNow[:] = xInitial[:] # Initialize the current solution , Set the initial solution to the current solution
    xBest[:] = xInitial[:] # Initialize the optimal solution , Set the current solution as the optimal solution
    fxNow = fxInitial # Set the objective function of the initial solution to the current value
    fxBest = fxInitial # Set the objective function of the current solution to the optimal value
    print('x_Initial:{:.6f},{:.6f},\tf(x_Initial):{:.6f}'.format(xInitial[0], xInitial[1], fxInitial))
    recordIter = [] # initialization , Number of external cycles
    recordFxNow = [] # initialization , The objective function value of the current solution
    recordFxBest = [] # initialization , The objective function value of the best solution
    recordPBad = [] # initialization , The acceptance probability of the inferior solution
    kIter = 0 # The number of iterations of the outer loop , The number of temperature states
    totalMar = 0 # A total of Markov Chain length
    totalImprove = 0 # fxBest The number of improvements
    nMarkov = meanMarkov # Fixed length Markov chain
    # ====== Start simulated annealing optimization ======
    # Outer loop , Until the current temperature reaches the end temperature
    tNow = tInitial # Initialize the current temperature (current temperature)
    while tNow >= tFinal: # Outer loop , Until the current temperature reaches the end temperature
        # At the current temperature , Do it enough times (nMarkov) To achieve thermal equilibrium
        kBetter = 0 # The number of times a good solution is obtained
        kBadAccept = 0 # The number of times a bad solution is accepted
        kBadRefuse = 0 # The number of times a bad solution is rejected
        # --- Inner loop , The number of cycles is Markov Chain length
        for k in range(nMarkov): # Inner loop , The number of cycles is Markov Chain length
            totalMar += 1 # total Markov Chain length counter
            # --- New solutions
            # New solutions ： A new solution is generated by random perturbation near the current solution , The new solution must be in [min,max] Within the scope of
            # programme 1： Only right n One of the variables is perturbed , Other n-1 Two variables remain unchanged
            xNew[:] = xNow[:]
            v = random.randint(0, nVar-1) # produce [0,nVar-1] Random number between
            xNew[v] = round(xNow[v] + scale * (xMax[v]-xMin[v]) * random.normalvariate(0, 1))
            # Satisfy that the decision variable is an integer , Use the simplest solution ： The resulting new solution is rounded to the nearest whole
            xNew[v] = max(min(xNew[v], xMax[v]), xMin[v]) # Make sure the new solution is [min,max] Within the scope of
            # --- Calculate the objective function and the energy difference
            # Call subfunction cal_Energy Calculate the objective function value of the new solution
            fxNew = cal_Energy(xNew, nVar, kIter)
            deltaE = fxNew - fxNow
            # --- Press Metropolis The criteria accept new interpretations
            # Accept judgment ： according to Metropolis The criteria decide whether to accept the new interpretation
            if fxNew < fxNow: # Better solution ： If the objective function of the new solution is better than the current solution , Then accept the new explanation
                accept = True
                kBetter += 1
            else: # Tolerance solution ： If the objective function of the new solution is worse than the current solution , The new solution is accepted with a certain probability
                pAccept = math.exp(-deltaE / tNow) # Calculate the state transition probability of the tolerant solution
                if pAccept > random.random():
                    accept = True # Accept the bad solution
                    kBadAccept += 1
                else:
                    accept = False # Refuse inferior solutions
                    kBadRefuse += 1
            # Save the new solution
            if accept == True: # If you accept the new explanation , The new solution is saved as the current solution
                xNow[:] = xNew[:]
                fxNow = fxNew
                if fxNew < fxBest: # If the objective function of the new solution is better than the optimal solution , Then the new solution is saved as the optimal solution
                    fxBest = fxNew
                    xBest[:] = xNew[:]
                    totalImprove += 1
                    scale = scale*0.99 # Variable search step size , Gradually reduce the search scope , Improve search accuracy
        # --- Data processing after the end of the inner loop
        # Complete the current temperature search , Save data and output
        pBadAccept = kBadAccept / (kBadAccept + kBadRefuse) # The acceptance probability of the inferior solution
        recordIter.append(kIter) # The current number of external loops
        recordFxNow.append(round(fxNow, 4)) # The objective function value of the current solution
        recordFxBest.append(round(fxBest, 4)) # The objective function value of the best solution
        recordPBad.append(round(pBadAccept, 4)) # The objective function value of the best solution
        if kIter%10 == 0: # Modular arithmetic , The remainder of the quotient
            print('i:{},t(i):{:.2f}, badAccept:{:.6f}, f(x)_best:{:.6f}'.\
                  format(kIter, tNow, pBadAccept, fxBest))
        # Slow down to a new temperature , The cooling curve ：T(k)=alfa*T(k-1)
        tNow = tNow * alfa
        kIter = kIter + 1
        fxBest = cal_Energy(xBest, nVar, kIter) # Because the penalty factor increases after iteration , Then we need to reconstruct the augmented objective function
    # ====== End the simulated annealing process ======
    print('improve:{:d}'.format(totalImprove))
    return kIter,xBest,fxBest,fxNow,recordIter,recordFxNow,recordFxBest,recordPBad

# Results check and output
def ResultOutput(cName,nVar,xBest,fxBest,kIter,recordFxNow,recordFxBest,recordPBad,recordIter):
    # ====== Check and output the optimization results ======
    fxCheck = cal_Energy(xBest, nVar, kIter)
    if abs(fxBest - fxCheck)>1e-3: # Test the objective function
        print("Error 2: Wrong total millage!")
        return
    else:
        print("\nOptimization by simulated annealing algorithm:")
        for i in range(nVar):
            print('\tx[{}] = {:.1f}'.format(i,xBest[i]))
        print('\n\tf(x) = {:.1f}'.format(cal_Energy(xBest,nVar,0)))
    return

# The main program
def main():
    # Parameter setting , Parameter definition of optimization problem , Parameter setting of simulated annealing algorithm
    [cName, nVar, xMin, xMax, tInitial, tFinal, alfa, meanMarkov, scale] = ParameterSetting()
    # print([nVar, xMin, xMax, tInitial, tFinal, alfa, meanMarkov, scale])
    # Simulated annealing algorithm
    [kIter,xBest,fxBest,fxNow,recordIter,recordFxNow,recordFxBest,recordPBad] \
    = OptimizationSSA(nVar,xMin,xMax,tInitial,tFinal,alfa,meanMarkov,scale)
    # print(kIter, fxNow, fxBest, pBadAccept)
    # Results check and output
    ResultOutput(cName, nVar,xBest,fxBest,kIter,recordFxNow,recordFxBest,recordPBad,recordIter)

if __name__ == '__main__':
    main()

x_Initial:4.000000,5.000000,	f(x_Initial):-85.000000
i:0,t(i):100.00, badAccept:0.890411, f(x)_best:-152.000000
i:10,t(i):81.71, badAccept:0.726027, f(x)_best:-98.000000
i:20,t(i):66.76, badAccept:0.571429, f(x)_best:-98.000000
i:30,t(i):54.55, badAccept:0.666667, f(x)_best:-98.000000
i:40,t(i):44.57, badAccept:0.688312, f(x)_best:-98.000000
i:50,t(i):36.42, badAccept:0.493506, f(x)_best:-98.000000
i:60,t(i):29.76, badAccept:0.580247, f(x)_best:-98.000000
i:70,t(i):24.31, badAccept:0.393258, f(x)_best:-98.000000
i:80,t(i):19.86, badAccept:0.494118, f(x)_best:-98.000000
i:90,t(i):16.23, badAccept:0.444444, f(x)_best:-98.000000
i:100,t(i):13.26, badAccept:0.337079, f(x)_best:-98.000000
i:110,t(i):10.84, badAccept:0.387097, f(x)_best:-98.000000
i:120,t(i):8.85, badAccept:0.115789, f(x)_best:-98.000000
i:130,t(i):7.23, badAccept:0.132653, f(x)_best:-98.000000
i:140,t(i):5.91, badAccept:0.357895, f(x)_best:-98.000000
i:150,t(i):4.83, badAccept:0.090909, f(x)_best:-98.000000
i:160,t(i):3.95,